In [10]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Define the dataset path
dataset_path = "/content/drive/MyDrive/CNN TECH-405/extracted/Laliga teams"

# Initialize lists to store images and labels
images = []
labels = []

# Loop through each team directory in the dataset path
for team in os.listdir(dataset_path):
    team_path = os.path.join(dataset_path, team)

    # Check if it is a directory
    if os.path.isdir(team_path):
        # Iterate over each image file in the team's directory
        for img_file in os.listdir(team_path):
            img_path = os.path.join(team_path, img_file)

            # Check if it's an image file (filtering out non-image files)
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                try:
                    # Open and resize the image
                    img = Image.open(img_path).convert('RGB')
                    img = img.resize((128, 128))
                    img_array = np.array(img)

                    # Append image and corresponding label
                    images.append(img_array)
                    labels.append(team)  # Use the team name as the label
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Debug: Check if images and labels have been loaded correctly
print(f"Loaded {len(images)} images with corresponding labels.")
if len(labels) > 0:
    print(f"Unique labels: {set(labels)}")

# Ensure the dataset is not empty
if len(images) == 0 or len(labels) == 0:
    raise ValueError("No images or labels were loaded. Please check your dataset path and files.")

# Normalize the images
X = images / 255.0

# Encode labels using one-hot encoding
label_encoder = LabelEncoder()
y = to_categorical(label_encoder.fit_transform(labels))

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Loaded 2000 images with corresponding labels.
Unique labels: {'Atletico Madrid', 'Villarreal', 'Girona', 'Osasuna', 'Celta Vigo', 'Athletic Bilbao', 'Leganes', 'Rayo Vallecano', 'Barcelona', 'Real Madrid', 'Real Sociedad', 'Espanyol', 'Las Palmas', 'Deportivo Alaves', 'Mallorca', 'Valencia', 'Real Betis', 'Sevilla', 'Real Valladolid', 'Getafe'}
Shape of X: (2000, 128, 128, 3)
Shape of y: (2000, 20)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │           5,140 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,521,172 (24.88 MB)

 Trainable params: 6,521,172 (24.88 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
Model: "sequential"

In [15]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

In [19]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    steps_per_epoch=len(X_train) // 32,
    validation_steps=len(X_val) // 32
)

Epoch 1/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.9743 - loss: 0.0833 - val_accuracy: 1.0000 - val_loss: 1.8276e-04
Epoch 2/30
 1/43 ━━━━━━━━━━━━━━━━━━━━ 36s 873ms/step - accuracy: 1.0000 - loss: 0.0051

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 1.0000 - loss: 0.0051 - val_accuracy: 1.0000 - val_loss: 2.0149e-04
Epoch 3/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.9844 - loss: 0.0567 - val_accuracy: 1.0000 - val_loss: 2.1741e-05
Epoch 4/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.9375 - loss: 0.2174 - val_accuracy: 1.0000 - val_loss: 3.2939e-05
Epoch 5/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.9842 - loss: 0.0556 - val_accuracy: 1.0000 - val_loss: 1.3213e-04
Epoch 6/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9688 - loss: 0.1141 - val_accuracy: 1.0000 - val_loss: 2.2242e-04
Epoch 7/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.9899 - loss: 0.0411 - val_accuracy: 1.0000 - val_loss: 0.0028
Epoch 8/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9688 - loss: 0.1106 - val_accuracy: 1.0000 - val_loss: 2.2321e-05
Epoch 9/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.9671 - loss: 0.1075 - val_accuracy: 1.

In [20]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Loss: {test_loss:.2f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - accuracy: 1.0000 - loss: 2.0684e-05
Test Accuracy: 1.00
Test Loss: 0.00


In [21]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = model.predict(X_test)                             # Predict the test set
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 367ms/step
                  precision    recall  f1-score   support

 Athletic Bilbao       1.00      1.00      1.00        14
 Atletico Madrid       1.00      1.00      1.00        13
       Barcelona       1.00      1.00      1.00        20
      Celta Vigo       1.00      1.00      1.00        16
Deportivo Alaves       1.00      1.00      1.00        17
        Espanyol       1.00      1.00      1.00        16
          Getafe       1.00      1.00      1.00         7
          Girona       1.00      1.00      1.00        19
      Las Palmas       1.00      1.00      1.00         8
         Leganes       1.00      1.00      1.00        19
        Mallorca       1.00      1.00      1.00        13
         Osasuna       1.00      1.00      1.00        11
  Rayo Vallecano       1.00      1.00      1.00        16
      Real Betis       1.00      1.00      1.00        18
     Real Madrid       1.00      1.00      1.00        13
   Real Sociedad       1.00   